https://syncandshare.lrz.de/dl/fiNDkYy3SNfH6M62mpfPV92c/processed.zip

In [0]:
import tensorflow as tf
import os

import gzip
import shutil
import pandas as pd

In [0]:
p = tf.keras.utils.get_file("processed.zip", origin="https://syncandshare.lrz.de/dl/fiNDkYy3SNfH6M62mpfPV92c/processed.zip", extract=True, cache_dir=".", cache_subdir="")
os.listdir(".")

5373952/Unknown - 14s 3us/step

['.config',
 'train_test_validation V0.1',
 'train_test_validation V0.2',
 'processed.zip',
 '.ipynb_checkpoints',
 'Data Splitting.ipynb',
 'sample_data']

In [0]:
data_train = pd.read_csv("train_test_validation V0.2/train/dataframe.csv", sep='|')
print(data_train.shape)
data_train.head()

(8823, 12)


,Author_ID,Author_name,Comment_number,Sentence_number,Domain_Relevance,Sentiment,Entity,Attribute,Sentence,Source_file,Annotator,Aspect
0,Justin-Ma,Justin Ma,521,1,0,NaN,NaN,NaN,Thanks for the thoughtful response.,quora.json,sumit,nan-nan
1,Justin-Ma,Justin Ma,521,2,0,NaN,NaN,NaN,I think we actually have a lot of common groun...,quora.json,sumit,nan-nan
2,Justin-Ma,Justin Ma,521,3,0,NaN,NaN,NaN,All I want to emphasize are my main points: Pr...,quora.json,sumit,nan-nan
3,Justin-Ma,Justin Ma,521,4,9,p,cg,pp,Industrialization is everything about producti...,quora.json,sumit,cg-pp
4,Justin-Ma,Justin Ma,521,5,0,NaN,NaN,NaN,Creating jobs at the expense of efficiency is ...,quora.json,sumit,nan-nan


In [0]:
data_train.Sentiment.value_counts(dropna=False)

NaN    4136
0      1812
p      1500
n      1375
Name: Sentiment, dtype: int64

In [0]:
data_train.Entity.value_counts(dropna=False)

NaN    4136
p      1635
g      1093
f       849
cp      298
c       289
gg      257
cf      204
cg       41
cc       21
Name: Entity, dtype: int64

In [0]:
data_train.Attribute.value_counts(dropna=False)

NaN    4136
g      1549
c       632
h       483
ll      333
q       311
p       296
e       235
s       199
t       172
pp      130
av      112
a       104
or       78
l        53
Name: Attribute, dtype: int64

In [0]:
data_train.Domain_Relevance.value_counts(dropna=False)

9    4687
0    4136
Name: Domain_Relevance, dtype: int64

In [0]:
od_entity_mapping = {
	'g': 'organic general',
	'p': 'organic products',
	'f': 'organic farmers',
	'c': 'organic companies',
	'cg': 'conventional general',
	'cp': 'conventional products',
	'cf': 'conventional farming',
	'cc': 'conventional companies',
	'gg': 'GMOs genetic engineering general'										
}
od_attribute_mapping = {
	'g': 'general',
	'p': 'price',
	't': 'taste',
	'q': 'Nutr. quality & freshness',
	's': 'safety',
	'h': 'healthiness',
	'c': 'chemicals pesticides',
	'll': 'label',
	'or': 'origin source',
	'l': 'local',
	'e': 'environment',
	'av': 'availability',
	'a': 'animal welfare',
	'pp': 'productivity'
}
od_sentiment_mapping = {
	'0': 'neutral',
	'p': 'positive',
	'n': 'negative'
}
od_coarse_entities = {
	'g': 'organic',
	'p': 'organic',
	'f': 'organic',
	'c': 'organic',

	'cg': 'conventional',
	'cp': 'conventional',
	'cf': 'conventional',
	'cc': 'conventional',

	'gg': 'GMO'
}

od_coarse_attributes = {
	'g': 'general',
	'p': 'price',
	
	't': 'experienced quality',
	'q': 'experienced quality',

	's': 'safety and healthiness',
	'h': 'safety and healthiness',
	'c': 'safety and healthiness',

	'll': 'trustworthy sources',
	'or': 'trustworthy sources',
	'l': 'trustworthy sources',
	'av': 'trustworthy sources',

	'e': 'environment',
	'a': 'environment',
	'pp': 'environment',
}


# filter / delete nan tests

In [0]:
data_train.head()

,Author_ID,Author_name,Comment_number,Sentence_number,Domain_Relevance,Sentiment,Entity,Attribute,Sentence,Source_file,Annotator,Aspect
0,Justin-Ma,Justin Ma,521,1,0,NaN,NaN,NaN,Thanks for the thoughtful response.,quora.json,sumit,nan-nan
1,Justin-Ma,Justin Ma,521,2,0,NaN,NaN,NaN,I think we actually have a lot of common groun...,quora.json,sumit,nan-nan
2,Justin-Ma,Justin Ma,521,3,0,NaN,NaN,NaN,All I want to emphasize are my main points: Pr...,quora.json,sumit,nan-nan
3,Justin-Ma,Justin Ma,521,4,9,p,cg,pp,Industrialization is everything about producti...,quora.json,sumit,cg-pp
4,Justin-Ma,Justin Ma,521,5,0,NaN,NaN,NaN,Creating jobs at the expense of efficiency is ...,quora.json,sumit,nan-nan


In [0]:
#f = data_train.dropna(subset=["Attribute"])
f = data_train[ data_train["Attribute"].notnull()]
f.head()

,Author_ID,Author_name,Comment_number,Sentence_number,Domain_Relevance,Sentiment,Entity,Attribute,Sentence,Source_file,Annotator,Aspect
3,Justin-Ma,Justin Ma,521,4,9,p,cg,pp,Industrialization is everything about producti...,quora.json,sumit,cg-pp
6,Justin-Ma,Justin Ma,521,7,9,0,g,g,'Organic' agriculture goes along with that the...,quora.json,sumit,g-g
18,Mick-Stute,Mick Stute,1286,1,9,n,cp,a,"It isn't innate in chickens to be ""disease pro...",quora.json,florian,cp-a
22,Mick-Stute,Mick Stute,1286,5,9,n,cp,a,What looks more healthy to you: factory farmed...,quora.json,florian,cp-a
27,Mick-Stute,Mick Stute,1286,10,9,p,cf,g,We currently only have 8 laying hens but even ...,quora.json,florian,cf-g


In [0]:
print(f.shape)
f["Attribute"].value_counts()

(4687, 12)


g     1549
c      632
h      483
ll     333
q      311
p      296
e      235
s      199
t      172
pp     130
av     112
a      104
or      78
l       53
Name: Attribute, dtype: int64

In [0]:
f = data_train[ data_train["Entity"].notnull()]
f.head()

,Author_ID,Author_name,Comment_number,Sentence_number,Domain_Relevance,Sentiment,Entity,Attribute,Sentence,Source_file,Annotator,Aspect
3,Justin-Ma,Justin Ma,521,4,9,p,cg,pp,Industrialization is everything about producti...,quora.json,sumit,cg-pp
6,Justin-Ma,Justin Ma,521,7,9,0,g,g,'Organic' agriculture goes along with that the...,quora.json,sumit,g-g
18,Mick-Stute,Mick Stute,1286,1,9,n,cp,a,"It isn't innate in chickens to be ""disease pro...",quora.json,florian,cp-a
22,Mick-Stute,Mick Stute,1286,5,9,n,cp,a,What looks more healthy to you: factory farmed...,quora.json,florian,cp-a
27,Mick-Stute,Mick Stute,1286,10,9,p,cf,g,We currently only have 8 laying hens but even ...,quora.json,florian,cf-g


In [0]:
print(f.shape)
f["Entity"].value_counts()

(4687, 12)


p     1635
g     1093
f      849
cp     298
c      289
gg     257
cf     204
cg      41
cc      21
Name: Entity, dtype: int64

In [0]:
f = data_train[ data_train["Domain_Relevance"].notnull()]
f.head()

,Author_ID,Author_name,Comment_number,Sentence_number,Domain_Relevance,Sentiment,Entity,Attribute,Sentence,Source_file,Annotator,Aspect
0,Justin-Ma,Justin Ma,521,1,0,NaN,NaN,NaN,Thanks for the thoughtful response.,quora.json,sumit,nan-nan
1,Justin-Ma,Justin Ma,521,2,0,NaN,NaN,NaN,I think we actually have a lot of common groun...,quora.json,sumit,nan-nan
2,Justin-Ma,Justin Ma,521,3,0,NaN,NaN,NaN,All I want to emphasize are my main points: Pr...,quora.json,sumit,nan-nan
3,Justin-Ma,Justin Ma,521,4,9,p,cg,pp,Industrialization is everything about producti...,quora.json,sumit,cg-pp
4,Justin-Ma,Justin Ma,521,5,0,NaN,NaN,NaN,Creating jobs at the expense of efficiency is ...,quora.json,sumit,nan-nan


In [0]:
print(f.shape)
f["Domain_Relevance"].value_counts()

(8823, 12)


9    4687
0    4136
Name: Domain_Relevance, dtype: int64